In [5]:
import sqlite3

conn = sqlite3.connect("cbt_data.db")
cursor = conn.cursor()

cursor.execute("SELECT id, title, content FROM cbt_entries")
rows = cursor.fetchall()

rows

[(1,
  'I’m not feeling well, but I don’t even know why.',
  'It’s okay to not have a clear reason for feeling low. Sometimes our mood drops because of underlying thought patterns or stress we haven’t consciously noticed. CBT helps identify these hidden thoughts—called automatic thoughts—that pop up and influence how we feel. For example, you might often think “I’m not good enough” without realizing it. These thoughts can silently weigh us down. Journaling or doing a thought record—where you write what you were thinking and how it made you feel—can start bringing more clarity and control.'),
 (2,
  'I feel tense all the time.',
  'Constant tension is a common response to chronic stress or anxiety. Your body is likely stuck in a “fight or flight” mode, even if there’s no danger. This physical state can be eased using progressive muscle relaxation—a CBT technique where you gently tense and release muscle groups one by one. This not only teaches your body how to relax, but also helps you 

In [6]:
!pip install ollama

In [ ]:
import ollama
EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
VECTOR_DB = [] 

for id_, title, body in rows:
    result = ollama.embed(model=EMBEDDING_MODEL, input=title)
    embedding = result['embedding'] if 'embedding' in result else result['embeddings']
    VECTOR_DB.append((id_, title, body, embedding))

In [9]:
VECTOR_DB

[(1,
  'I’m not feeling well, but I don’t even know why.',
  'It’s okay to not have a clear reason for feeling low. Sometimes our mood drops because of underlying thought patterns or stress we haven’t consciously noticed. CBT helps identify these hidden thoughts—called automatic thoughts—that pop up and influence how we feel. For example, you might often think “I’m not good enough” without realizing it. These thoughts can silently weigh us down. Journaling or doing a thought record—where you write what you were thinking and how it made you feel—can start bringing more clarity and control.',
  [[0.005175356,
    -0.013919503,
    0.025335893,
    -0.022613525,
    0.0016265142,
    0.0057977797,
    0.012035771,
    0.025437173,
    0.01989817,
    -0.04667434,
    0.011794648,
    0.009904333,
    -0.052050542,
    -0.0073462604,
    0.014347049,
    0.07653895,
    0.0323353,
    -0.045167513,
    -0.010938081,
    0.009294554,
    0.022423835,
    0.001971145,
    -0.029934896,
    -

In [29]:
import numpy as np
import faiss

raw_embedding = VECTOR_DB[0][3]
if isinstance(raw_embedding[0], list):  # nested
    raw_embedding = raw_embedding[0]
dimension = len(raw_embedding)
index = faiss.IndexFlatL2(dimension)

id_list, body_list = [], []

for id_, title, body, embedding in VECTOR_DB:
    if isinstance(embedding[0], list): 
        embedding = embedding[0]
    embedding = np.array(embedding, dtype='float32').reshape(1, -1)  

    index.add(embedding)
    id_list.append(id_)
    body_list.append(body)

In [41]:
query = "I'm feeling hopeless"
query_vector = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings']
D, I = index.search(np.array([query_vector[0]], dtype='float32'), k=3)

# Show results
for idx in I[0]:
    print(f"Matched ID: {id_list[idx]}")
    print(f"Response Chunk: {body_list[idx]}")

Matched ID: 20
Response Chunk: When you’re exhausted mentally and emotionally, giving up can seem like the only option. But that feeling isn’t permanent. CBT teaches that emotions—no matter how intense—can and do change. You can start small: reach out, rest, write one thought down. You don’t have to climb the whole mountain today—just take one step. And if you're truly overwhelmed, please connect with a mental health line or professional. Help exists, and things can improve—especially when you let someone walk with you.
Matched ID: 43
Response Chunk: Feeling like a failure doesn’t mean you are one. CBT helps by showing that our thoughts aren’t facts. Maybe you didn’t meet a goal—but that’s an event, not your identity. Reframe with: “This didn’t go as planned, but I’m learning.” Keep a list of things you’ve done well, no matter how small. Self-worth grows not from perfection, but from persistence.
Matched ID: 59
Response Chunk: Grief can make the future feel blank or hopeless. CBT works

In [53]:
I

array([[19, 42, 58]])

In [48]:
faiss.write_index(index, "index.faiss")

In [49]:
import json

with open("meta.json", "w") as f:
    json.dump({
        "id_list": id_list,
        "body_list": body_list
    }, f)